<h1>Toronto Data Project</h1>

<h2>Section 1</h2>
<h3>Please scroll down for the second and third scetions</h3>

In [158]:
#importing the required libraries
from bs4 import BeautifulSoup
from urllib import request
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [159]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [160]:
page = request.urlopen(url)
page

<h3>The Beautiful Soup package is used to parse through the Wikipedia page</h3>

In [161]:
soup = BeautifulSoup(page, 'lxml')

In [162]:
Toronto_Table = soup.find('table', class_='wikitable sortable')

In [163]:
Toronto_Table_List = []

In [164]:
for element in Toronto_Table.find_all('tr'):
    Toronto_Table_List.append(list(element))
del(Toronto_Table_List)[0]

In [165]:
A = []
B = []
C = []

for element in Toronto_Table_List:
    A.append(element[1].find(text=True))
    B.append(element[3].find(text=True))
    C.append(element[5].find(text=True))

In [166]:
Toronto_Data = pd.DataFrame(A, columns=['Postal Code'])
Toronto_Data['Borough'] = B
Toronto_Data['Neighbourhood'] = C

In [167]:
for element in Toronto_Data.columns.to_list():
    Toronto_Data[element] = Toronto_Data[element].str.strip()
    Toronto_Data[element] = Toronto_Data[element].replace('Not assigned',np.nan)
Toronto_Data

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,NaN,NaN
176,M6Z,NaN,NaN
177,M7Z,NaN,NaN
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


<h3>Cleaned Dataframe</h3>
All Boroughs that did not have values were removed and Neighbourhoods without values were assigned their corresponding Borough name.

In [168]:
Toronto_Data.dropna(subset=['Borough'],axis=0,inplace=True)
Toronto_Data.reset_index(inplace=True)
Toronto_Data.drop('index',axis=1)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [169]:
Toronto_Data.head(10)

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [170]:
Toronto_Data.shape

(103, 4)

<h2>Section 2</h2>
<h3>Please scroll down for the third section</h3>
The Coordinates are obtained from the CSV file provided.

In [171]:
GeoCod = pd.read_csv('Geospatial_Coordinates.csv')
GeoCod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [172]:
#help(pd.merge)

In [173]:
New_Toronto_Data = pd.merge(Toronto_Data,GeoCod,left_on='Postal Code',right_on='Postal Code')

In [174]:
New_Toronto_Data.drop('index',axis=1,inplace=True)

In [175]:
New_Toronto_Data.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2>Section 3</h2>
Calls are made to the foursquare API and a KMeans clustering algorithm is run

<h3>Visualization of the neighbourhoods in Toronto</h3>

In [176]:
# create map of New York using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(New_Toronto_Data['Latitude'], New_Toronto_Data['Longitude'], New_Toronto_Data['Borough'], New_Toronto_Data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>In this section all the Neighbourhoods in the York section of Toronto will be explored</h3>
This includes:
<li>North York
<li>East York 
<li>York

In [177]:
YorkToronto_Data = New_Toronto_Data[New_Toronto_Data['Borough'].str.contains('York',na=False)].reset_index(drop=True)

In [178]:
YorkToronto_Data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M3C,North York,Don Mills,43.725900,-79.340923
7,M4C,East York,Woodbine Heights,43.695344,-79.318389
8,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
9,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512


In [179]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(YorkToronto_Data['Latitude'], YorkToronto_Data['Longitude'], YorkToronto_Data['Borough'], YorkToronto_Data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [180]:
address = 'York, Toronto'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6896191, -79.479188.


In [182]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The most common venues within a 1000m radius are generated using the Foursquare API

In [183]:
YorkToronto_Venues = getNearbyVenues(names=CentralToronto_Data['Neighbourhood'],
                                   latitudes=CentralToronto_Data['Latitude'],
                                   longitudes=CentralToronto_Data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [184]:
YorkToronto_Venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
...,...,...,...,...,...,...,...
1013,"Willowdale, Willowdale West",43.782736,-79.442259,Northview Variety,43.775252,-79.440701,Convenience Store
1014,"Willowdale, Willowdale West",43.782736,-79.442259,Got to half it Baby!,43.778242,-79.450862,Baby Store
1015,"Willowdale, Willowdale West",43.782736,-79.442259,Belle Restaurant,43.774325,-79.442651,Eastern European Restaurant
1016,"Willowdale, Willowdale West",43.782736,-79.442259,Happy Bagel Bakery,43.774008,-79.442697,Bakery


In [185]:
YorkToronto_Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,15,15,15,15,15,15
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
Caledonia-Fairbanks,20,20,20,20,20,20
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",16,16,16,16,16,16
Don Mills,73,73,73,73,73,73
Downsview,69,69,69,69,69,69
"East Toronto, Broadview North (Old East York)",98,98,98,98,98,98
"Fairview, Henry Farm, Oriole",44,44,44,44,44,44


In [186]:
print('There are ',YorkToronto_Venues['Venue Category'].nunique(),' unique venue categories')

There are  183  unique venue categories


In [187]:
YorkToronto_OneHot = pd.get_dummies(YorkToronto_Venues[['Venue Category']], prefix='', prefix_sep="")
YorkToronto_OneHot['Neighbourhood'] = YorkToronto_Venues['Neighborhood']

fixed = [YorkToronto_OneHot.columns[-1]] + list(YorkToronto_OneHot.columns[:-1])
YorkToronto_OneHot = YorkToronto_OneHot[fixed]

Onehot encoding is used to make the clustering possible 

In [189]:
YorkToronto_OneHot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
YorkToronto_OneHot.shape

(1018, 184)

In [191]:
YorkToronto_Grouped = YorkToronto_OneHot.groupby('Neighbourhood').mean().reset_index()

In [192]:
YorkToronto_Grouped 

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.024390,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.02439,0.000000,0.000000
3,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.050000,0.000000
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.062500,0.00000,0.000000,0.000000
5,Don Mills,0.000000,0.000000,0.000000,0.013699,0.013699,0.00,0.027397,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.013699,0.000000
6,Downsview,0.000000,0.000000,0.014493,0.014493,0.000000,0.00,0.000000,0.028986,0.000000,...,0.000000,0.028986,0.000000,0.0000,0.072464,0.000000,0.000000,0.00000,0.000000,0.000000
7,"East Toronto, Broadview North (Old East York)",0.000000,0.000000,0.000000,0.020408,0.000000,0.00,0.010204,0.000000,0.000000,...,0.000000,0.010204,0.000000,0.0000,0.010204,0.000000,0.010204,0.00000,0.000000,0.000000
8,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.000000,0.022727,0.000000,0.00,0.022727,0.000000,0.000000,...,0.000000,0.000000,0.022727,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
9,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030303,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


In [193]:
YorkToronto_Grouped [YorkToronto_Grouped ['Neighbourhood']=='Davisville'].T.reset_index()

,index
0,Neighbourhood
1,Accessories Store
2,Afghan Restaurant
3,Airport
4,American Restaurant
...,...
179,Warehouse Store
180,Wine Shop
181,Wings Joint
182,Women's Store


In [194]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [195]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = YorkToronto_Grouped ['Neighbourhood']

for ind in np.arange(YorkToronto_Grouped .shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(YorkToronto_Grouped .iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Pharmacy,Mediterranean Restaurant,Shopping Mall,Bridal Shop,Sandwich Place,Dog Run,Restaurant
1,Bayview Village,Grocery Store,Gas Station,Bank,Japanese Restaurant,Park,Trail,Restaurant,Café,Intersection,Skating Rink
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Bank,Pizza Place,Sandwich Place,Restaurant,Liquor Store,Intersection,Skating Rink
3,Caledonia-Fairbanks,Pharmacy,Park,Bus Stop,Bank,Cosmetics Shop,Pizza Place,Coffee Shop,Discount Store,Mexican Restaurant,Falafel Restaurant
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Furniture / Home Store,Discount Store,Grocery Store,Sandwich Place,Dessert Shop,Italian Restaurant,Gas Station,Bar,Coffee Shop,Video Store
5,Don Mills,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Pizza Place,Mobile Phone Shop
6,Downsview,Coffee Shop,Vietnamese Restaurant,Hotel,Gas Station,Park,Grocery Store,Pizza Place,Athletics & Sports,Shopping Mall,Sandwich Place
7,"East Toronto, Broadview North (Old East York)",Coffee Shop,Café,Greek Restaurant,Ethiopian Restaurant,Beer Bar,Fast Food Restaurant,Pizza Place,Pharmacy,Thai Restaurant,Gym
8,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Sandwich Place,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Restaurant,Movie Theater
9,Glencairn,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Park,Restaurant,Mobile Phone Shop,Pub,Playground,Pizza Place


<h2>Clustering</h2>

In [208]:
YorkToronto_Cluster = YorkToronto_Grouped .drop('Neighbourhood', axis=1)
YorkToronto_Cluster.head()

,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,...,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.00000,0.00,0.0
1,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.00000,0.00,0.0
2,0.0,0.0,0.0,0.02439,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.02439,0.00,0.0
3,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0000,0.00000,0.05,0.0
4,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0625,0.0,0.0,0.0625,0.00000,0.00,0.0


In [229]:
Num_Clusters = 3
kmeans = KMeans(n_clusters=Num_Clusters,random_state=False).fit(YorkToronto_Cluster)
kmeans.labels_[0:10]

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 1])

In [230]:
New_df = YorkToronto_Data.drop(['Postal Code','Borough'],axis=1)

In [231]:
New_df = New_df.merge(neighborhoods_venues_sorted,how='right',left_on='Neighbourhood',right_on='Neighborhood')

In [232]:
New_df = New_df.drop_duplicates(subset='Neighbourhood').reset_index(drop=True)

In [233]:
New_df = New_df.drop('Neighborhood',axis=1)

In [234]:
New_df['Cluster Labels'] = kmeans.labels_

In [235]:
New_df.head()

,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Parkwoods,43.753259,-79.329656,Park,Pharmacy,Bus Stop,Convenience Store,Shopping Mall,Road,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,0
1,Victoria Village,43.725882,-79.315572,Coffee Shop,Pizza Place,Golf Course,French Restaurant,Intersection,Café,Portuguese Restaurant,Men's Store,Gym / Fitness Center,Grocery Store,1
2,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Furniture / Home Store,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Athletics & Sports,Dessert Shop,Sushi Restaurant,0
3,Don Mills,43.745906,-79.352188,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Pizza Place,Mobile Phone Shop,1
4,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Gym / Fitness Center,Pizza Place,Brewery,Bakery,Soccer Stadium,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy,0


<h3>Visualization of the Clusters in the York part of Toronto</h3>

In [236]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Num_Clusters)
ys = [i + x + (i*x)**2 for i in range(Num_Clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(New_df['Latitude'], New_df['Longitude'], New_df['Neighbourhood'], New_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Cluster Results<h3>

0 First Cluster

In [241]:
First = New_df.loc[New_df['Cluster Labels'] == 0, New_df.columns[[0] + list(range(3, New_df.shape[1]-1))]]
First.reset_index(drop=True)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Pharmacy,Bus Stop,Convenience Store,Shopping Mall,Road,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop
1,"Lawrence Manor, Lawrence Heights",Furniture / Home Store,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Athletics & Sports,Dessert Shop,Sushi Restaurant
2,"Parkview Hill, Woodbine Gardens",Gym / Fitness Center,Pizza Place,Brewery,Bakery,Soccer Stadium,Gastropub,Intersection,Fast Food Restaurant,Pet Store,Pharmacy
3,Glencairn,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Park,Restaurant,Mobile Phone Shop,Pub,Playground,Pizza Place
4,Woodbine Heights,Park,Coffee Shop,Athletics & Sports,Sandwich Place,Skating Rink,Pizza Place,Liquor Store,Bus Line,Restaurant,Convenience Store
5,Humewood-Cedarvale,Pizza Place,Bagel Shop,Convenience Store,Coffee Shop,Grocery Store,Field,Frozen Yogurt Shop,Deli / Bodega,Optical Shop,Middle Eastern Restaurant
6,Caledonia-Fairbanks,Pharmacy,Park,Bus Stop,Bank,Cosmetics Shop,Pizza Place,Coffee Shop,Discount Store,Mexican Restaurant,Falafel Restaurant
7,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Pharmacy,Mediterranean Restaurant,Shopping Mall,Bridal Shop,Sandwich Place,Dog Run,Restaurant
8,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Sandwich Place,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Bank,Bakery,Restaurant,Movie Theater
9,"Northwood Park, York University",Pizza Place,Furniture / Home Store,Coffee Shop,Restaurant,Caribbean Restaurant,Massage Studio,Metro Station,Middle Eastern Restaurant,Fast Food Restaurant,Miscellaneous Shop


1 Second Cluster

In [242]:
Second = New_df.loc[New_df['Cluster Labels'] == 1, New_df.columns[[0] + list(range(3, New_df.shape[1]-1))]]
Second.reset_index(drop=True)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Victoria Village,Coffee Shop,Pizza Place,Golf Course,French Restaurant,Intersection,Café,Portuguese Restaurant,Men's Store,Gym / Fitness Center,Grocery Store
1,Don Mills,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Bank,Supermarket,Burger Joint,Asian Restaurant,Pizza Place,Mobile Phone Shop
2,Leaside,Furniture / Home Store,Sporting Goods Shop,Coffee Shop,Grocery Store,Electronics Store,Department Store,Brewery,Sandwich Place,Burger Joint,Restaurant
3,Hillcrest Village,Park,Pharmacy,Coffee Shop,Pizza Place,Shopping Mall,Ice Cream Shop,Intersection,Korean Restaurant,Convenience Store,Chinese Restaurant
4,Thorncliffe Park,Coffee Shop,Grocery Store,Indian Restaurant,Shopping Mall,Supermarket,Afghan Restaurant,Brewery,Sandwich Place,Turkish Restaurant,Burger Joint
5,Bayview Village,Grocery Store,Gas Station,Bank,Japanese Restaurant,Park,Trail,Restaurant,Café,Intersection,Skating Rink
6,"North Park, Maple Leaf Park, Upwood Park",Coffee Shop,Convenience Store,Gas Station,Park,Chinese Restaurant,Dim Sum Restaurant,Mediterranean Restaurant,Bakery,Pizza Place,Athletics & Sports
7,"Runnymede, The Junction North",Coffee Shop,Pizza Place,Brewery,Park,Gas Station,Bus Line,Athletics & Sports,Convenience Store,Beer Store,Discount Store
8,Weston,Pizza Place,Coffee Shop,Train Station,Grocery Store,Bank,Café,Diner,Sandwich Place,Skating Rink,Convenience Store
9,York Mills West,Park,Coffee Shop,Restaurant,Playground,Tennis Court,Golf Course,Gas Station,French Restaurant,Dog Run,Dentist's Office


2 Third Cluster

In [243]:
Third = New_df.loc[New_df['Cluster Labels'] == 2, New_df.columns[[0] + list(range(3, New_df.shape[1]-1))]]
Third.reset_index(drop=True)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Willowdale, Willowdale West",Pharmacy,Convenience Store,Baby Store,Park,Coffee Shop,Eastern European Restaurant,Bus Line,Bank,Bakery,Pizza Place


Three clusters were generated from the KMeans algorithm
<li> The First Cluster has 20 Neighbourhoods. This cluster seems to be majorly made up of banks, parks, coffee shops and Cafes.
<li> The second cluster is made up of 10 Neighbourhoods. This cluster seems very similar to the first as they both have lots of coffee shops and parks, but this cluster has a variety of exotic resturants such as French restaurants, Chinese restaurants, Vietnamese Restaurants, Mediterranean Restaurants
<li>The third cluster has just one Neighbourhood. It seems like an outlier
